# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

~~[SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[ResNet34](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)~~
~~[PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)~~
~~[InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[BNInception](https://github.com/Cadene/pretrained-models.pytorch#bninception)~~
~~[InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)~~
~~[Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)~~
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet131](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)


 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc
from tqdm import tqdm

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import *
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
2,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
3,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
4,49850884a00703afe5aab78c3ce074d2d4acae30.jpg,../data/cleaned/none/49850884a00703afe5aab78c3...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 46/46 [00:09<00:00,  5.04it/s]


Image reading complete.
Image array shape: (299, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--lr", type=float, default=LEARNING_RATE)
    parser.add_argument("--weight_decay", type=float, default=WEIGHT_DECAY)
    parser.add_argument("--epsilon", type=float, default=EPSILON)
    parser.add_argument("--amsgrad", type=bool, default=AMSGRAD)
    parser.add_argument("--betas", default=BETAS)
    parser.add_argument("--eta_min", type=float, default=ETA_MIN)
    parser.add_argument("--t_max", type=float, default=T_MAX)
    parser.add_argument("--t_mult", type=float, default=T_MULT)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = se_resnext101_32x4d()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.hparams.lr, 
            betas=self.hparams.betas, 
            eps=self.hparams.epsilon, 
            weight_decay=self.hparams.weight_decay,
            amsgrad=self.hparams.amsgrad
        )
        self.scheduler = WarmRestart(
            self.optimizer, 
            T_max=self.hparams.t_max, 
            T_mult=self.hparams.t_mult, 
            eta_min=self.hparams.eta_min
        )
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=self.hparams.t_mult)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())

        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()
torch.cuda.empty_cache()

log_notes = "t_mult increased to 6 from 5"

# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Log parameters
logger.info(f"backbone: {hparams.backbone}")
logger.info(f"device_name: {hparams.device_name}")
logger.info(f"gpus: {hparams.gpus}")
logger.info(f"n_workers: {hparams.n_workers}")
logger.info(f"image_size: {hparams.image_size}")
logger.info(f"seed: {hparams.seed}")
logger.info(f"min_epochs: {hparams.min_epochs}")
logger.info(f"max_epochs: {hparams.max_epochs}")
logger.info(f"patience: {hparams.patience}")
logger.info(f"train_batch_size: {hparams.train_batch_size}")
logger.info(f"val_batch_size: {hparams.val_batch_size}")
logger.info(f"n_splits: {hparams.n_splits}")
logger.info(f"test_size: {hparams.test_size}")
logger.info(f"learning rate: {hparams.lr}")
logger.info(f"weight_decay: {hparams.weight_decay}")
logger.info(f"epsilon: {hparams.epsilon}")
logger.info(f"amsgrad: {hparams.amsgrad}")
logger.info(f"betas: {hparams.betas}")
logger.info(f"precision: {hparams.precision}")
logger.info(f"gradient_clip_val: {hparams.gradient_clip_val}")
logger.info(f"eta_min: {hparams.eta_min}")
logger.info(f"t_max: {hparams.t_max}")
logger.info(f"t_mult: {hparams.t_mult}")
logger.info(f"log_dir: {hparams.log_dir}")
logger.info(f"log_name: {hparams.log_name}")

# Log any notes if they exist
if "log_notes" in locals():
    logger.info(f"Notes: {log_notes}")


# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
val_loss_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_loss", 
        patience=hparams.patience, 
        mode="min", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
            
    # Save val scores
    val_loss_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
val_loss_scores = [i.item() for i in val_loss_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [val_loss_scores],
            "mean_score": [np.mean(val_loss_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {val_loss_scores}")
logger.info("Training complete.")

[2022-12-08 09:36:07] 3687493274.py[  11] : INFO  backbone: se_resnext101_32x4d
[2022-12-08 09:36:07] 3687493274.py[  12] : INFO  device_name: NVIDIA GeForce RTX 3090
[2022-12-08 09:36:07] 3687493274.py[  13] : INFO  gpus: [0]
[2022-12-08 09:36:07] 3687493274.py[  14] : INFO  n_workers: 128
[2022-12-08 09:36:07] 3687493274.py[  15] : INFO  image_size: [1024, 1024]
[2022-12-08 09:36:07] 3687493274.py[  16] : INFO  seed: 14
[2022-12-08 09:36:07] 3687493274.py[  17] : INFO  min_epochs: 50
[2022-12-08 09:36:07] 3687493274.py[  18] : INFO  max_epochs: 100
[2022-12-08 09:36:07] 3687493274.py[  19] : INFO  patience: 11
[2022-12-08 09:36:07] 3687493274.py[  20] : INFO  train_batch_size: 4
[2022-12-08 09:36:07] 3687493274.py[  21] : INFO  val_batch_size: 4
[2022-12-08 09:36:07] 3687493274.py[  22] : INFO  n_splits: 3
[2022-12-08 09:36:07] 3687493274.py[  23] : INFO  test_size: 0.1
[2022-12-08 09:36:07] 3687493274.py[  24] : INFO  learning rate: 0.0001
[2022-12-08 09:36:07] 3687493274.py[  25] :

[2022-12-08 09:59:22] 1068281035.py[  95] : INFO  0-37 | lr : 0.000087 | val_loss : 1.4747 | val_roc_auc : 0.7765 | data_load_times : 45.36 | batch_run_times : 45.90
[2022-12-08 09:59:58] 1068281035.py[  95] : INFO  0-38 | lr : 0.000085 | val_loss : 1.4120 | val_roc_auc : 0.7850 | data_load_times : 45.80 | batch_run_times : 46.38
[2022-12-08 10:00:36] 1068281035.py[  95] : INFO  0-39 | lr : 0.000084 | val_loss : 1.3271 | val_roc_auc : 0.8353 | data_load_times : 46.07 | batch_run_times : 46.62
[2022-12-08 10:01:13] 1068281035.py[  95] : INFO  0-40 | lr : 0.000083 | val_loss : 1.5139 | val_roc_auc : 0.8033 | data_load_times : 45.05 | batch_run_times : 45.61
[2022-12-08 10:01:49] 1068281035.py[  95] : INFO  0-41 | lr : 0.000082 | val_loss : 1.4819 | val_roc_auc : 0.8163 | data_load_times : 46.24 | batch_run_times : 46.81
[2022-12-08 10:02:26] 1068281035.py[  95] : INFO  0-42 | lr : 0.000080 | val_loss : 1.6713 | val_roc_auc : 0.7933 | data_load_times : 35.83 | batch_run_times : 36.65
[202

[2022-12-08 10:30:04] 1068281035.py[  95] : INFO  1-34 | lr : 0.000090 | val_loss : 1.6960 | val_roc_auc : 0.7690 | data_load_times : 44.58 | batch_run_times : 45.14
[2022-12-08 10:30:43] 1068281035.py[  95] : INFO  1-35 | lr : 0.000089 | val_loss : 1.7396 | val_roc_auc : 0.7498 | data_load_times : 46.15 | batch_run_times : 46.77
[2022-12-08 10:31:21] 1068281035.py[  95] : INFO  1-36 | lr : 0.000088 | val_loss : 1.5753 | val_roc_auc : 0.8063 | data_load_times : 45.83 | batch_run_times : 46.66
[2022-12-08 10:32:00] 1068281035.py[  95] : INFO  1-37 | lr : 0.000087 | val_loss : 1.6585 | val_roc_auc : 0.7722 | data_load_times : 44.80 | batch_run_times : 45.35
[2022-12-08 10:32:39] 1068281035.py[  95] : INFO  1-38 | lr : 0.000085 | val_loss : 1.5753 | val_roc_auc : 0.7776 | data_load_times : 46.27 | batch_run_times : 47.13
[2022-12-08 10:33:17] 1068281035.py[  95] : INFO  1-39 | lr : 0.000084 | val_loss : 1.6590 | val_roc_auc : 0.7835 | data_load_times : 46.41 | batch_run_times : 46.95
[202

[2022-12-08 10:57:23] 1068281035.py[  95] : INFO  2-25 | lr : 0.000097 | val_loss : 1.5535 | val_roc_auc : 0.8218 | data_load_times : 48.53 | batch_run_times : 49.15
[2022-12-08 10:58:03] 1068281035.py[  95] : INFO  2-26 | lr : 0.000097 | val_loss : 1.5443 | val_roc_auc : 0.8217 | data_load_times : 45.57 | batch_run_times : 46.18
[2022-12-08 10:58:44] 1068281035.py[  95] : INFO  2-27 | lr : 0.000096 | val_loss : 1.5695 | val_roc_auc : 0.8047 | data_load_times : 48.36 | batch_run_times : 48.92
[2022-12-08 10:59:24] 1068281035.py[  95] : INFO  2-28 | lr : 0.000095 | val_loss : 1.5927 | val_roc_auc : 0.7822 | data_load_times : 47.85 | batch_run_times : 48.38
[2022-12-08 11:00:05] 1068281035.py[  95] : INFO  2-29 | lr : 0.000094 | val_loss : 1.5698 | val_roc_auc : 0.7734 | data_load_times : 46.66 | batch_run_times : 47.21
[2022-12-08 11:00:45] 1068281035.py[  95] : INFO  2-30 | lr : 0.000094 | val_loss : 1.4674 | val_roc_auc : 0.8141 | data_load_times : 46.33 | batch_run_times : 46.87
[202

## Validation Inference

In [7]:
# Get model run path and define chosen fold
log_dir = "../logs/logs"
#model_run = "2022_11_08_14:57:52" # manually choose model path
model_run = hparams.log_name
model_run_path = os.path.join(log_dir, model_run)
#best_fold = 1 # manually choose model path
best_fold = val_loss_scores.index(min(val_loss_scores))

# Get best model for chosen fold
model_run_dir = os.listdir(model_run_path)
model_folds = [i for i in model_run_dir if i.startswith(f"fold={best_fold}")]
model_folds_scores = [float(i.split("val_loss=")[1].split("-")[0]) for i in model_folds]
model_name = model_folds[model_folds_scores.index(min(model_folds_scores))]
model_path = os.path.join(model_run_path, model_name)

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(model_path)["state_dict"]
)
model.eval()

# Retrieve validation indices for chosen fold
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    if fold_i == best_fold:
        break

# Select fold validation images
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Create predictions looped by batch
counter = 0
val_i_batch = []
val_idx_batch = []
scores_df = pd.DataFrame()

for i, idx in tqdm(enumerate(val_index)):
    counter += 1
    val_i_batch.append(i) # arrays don't preserve index so need ordered index values
    val_idx_batch.append(idx) # for preserved index
    
    # Run inference for val_batch_size
    if counter == hparams.val_batch_size:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        # Reset counter and batch
        counter = 0
        val_i_batch = []
        val_idx_batch = []
        
    # Run inference for remaining batch
    elif idx == val_index[-1]:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

        
# Write predictions to log
scores_df.to_csv(
    os.path.join(model_run_path, f"{model_run}_preds_fold_{best_fold}.csv"),
    index=False
)

100it [07:22,  4.43s/it]


In [8]:
scores_df

,img_path,label,ant,bedbug,bee,horsefly,mite,mosquito,none,tick
0,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,0.124998,0.127505,0.138024,0.129489,0.149257,0.112448,0.096403,0.121876
1,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,0.119071,0.127625,0.141309,0.128924,0.150748,0.114408,0.098422,0.119494
2,../data/cleaned/none/6c68654ed15dc485d527dd85f...,none,0.121919,0.118456,0.141615,0.136042,0.145989,0.117594,0.097614,0.120771
3,../data/cleaned/none/f64f47a69e72ce97b3ae2b07e...,none,0.123417,0.127582,0.138511,0.128951,0.155995,0.108925,0.097903,0.118715
4,../data/cleaned/none/b8f71c61c19392c3cb24ebc54...,none,0.122788,0.129511,0.139414,0.127349,0.153652,0.111325,0.097411,0.118549
...,...,...,...,...,...,...,...,...,...,...
95,../data/cleaned/ant/9a0a79963955f8ac3e6de74750...,ant,0.123945,0.127328,0.136644,0.129611,0.151594,0.113221,0.096889,0.120767
96,../data/cleaned/ant/f54a2ecb91d8d3d0935d518d81...,ant,0.119437,0.126298,0.142780,0.130300,0.153670,0.113144,0.095956,0.118415
97,../data/cleaned/ant/06443ff82c28fd0d52f62868d8...,ant,0.124024,0.123984,0.137007,0.133916,0.151467,0.113509,0.096801,0.119291
98,../data/cleaned/ant/ea946cc42f2daadb6eca6aac12...,ant,0.125333,0.127760,0.138207,0.128275,0.149787,0.113549,0.096532,0.120557


## Validation Analysis

In [9]:
print(f"{len(scores_df['img_path'].unique())} unique image paths.")

100 unique image paths.


In [10]:
print("Validation label counts:")
print(scores_df["label"].value_counts())

Validation label counts:
bedbug      20
tick        19
mosquito    15
ant         15
none         9
horsefly     8
mite         7
bee          7
Name: label, dtype: int64


In [11]:
print("Validation prediction counts:")
print(
    pd.melt(
        scores_df,
        id_vars=["img_path", "label"],
        value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
        var_name="pred_label",
        value_name="pred_prob"
    ).sort_values(["img_path", "pred_prob"], ascending=False) \
    .groupby(["img_path", "label"]).first()["pred_label"] \
    .value_counts()
)

Validation prediction counts:
mite    100
Name: pred_label, dtype: int64


In [12]:
# Probability stats by label
pd.concat(
    [
        pd.DataFrame(scores_df.iloc[:, 2:].mean(), columns=["mean"]),
        pd.DataFrame(scores_df.iloc[:, 2:].std(), columns=["std"]),
        pd.DataFrame(scores_df.iloc[:, 2:].min(), columns=["min"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.25)),
        pd.DataFrame(scores_df.iloc[:, 2:].median(), columns=["median"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.75)),
        pd.DataFrame(scores_df.iloc[:, 2:].max(), columns=["max"]),
        pd.DataFrame(scores_df.iloc[:, 2:].max() - scores_df.iloc[:, 2:].min(), columns=["range"])
    ], 
    axis=1
)

,mean,std,min,0.25,median,0.75,max,range
ant,0.123860,0.002391,0.118318,0.122737,0.124608,0.125526,0.127672,0.009354
bedbug,0.125951,0.002448,0.118164,0.124688,0.126599,0.127593,0.130038,0.011874
bee,0.138538,0.002170,0.134231,0.137313,0.138415,0.139370,0.143347,0.009117
horsefly,0.131333,0.003211,0.126209,0.129120,0.130857,0.132999,0.140255,0.014046
mite,0.151062,0.002139,0.145761,0.149355,0.151096,0.152340,0.159071,0.013310
mosquito,0.112704,0.001977,0.106808,0.111812,0.112896,0.114012,0.117594,0.010786
none,0.096256,0.000840,0.093781,0.095727,0.096318,0.096773,0.098422,0.004641
tick,0.120296,0.001496,0.116227,0.119460,0.120503,0.121232,0.124697,0.008471


In [13]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).pivot_table(
    index=["label"],
    columns=["pred_label"],
    aggfunc="mean"
)

pred_prob                                                    \
pred_label       ant    bedbug       bee  horsefly      mite  mosquito   
label                                                                    
ant         0.124070  0.126414  0.138031  0.130843  0.151572  0.112613   
bedbug      0.124016  0.125785  0.138380  0.132091  0.151048  0.112173   
bee         0.124523  0.123855  0.136868  0.134502  0.152375  0.110869   
horsefly    0.123396  0.126054  0.138131  0.132124  0.152386  0.111753   
mite        0.123323  0.126742  0.139457  0.129705  0.151186  0.113194   
mosquito    0.124443  0.125687  0.138198  0.131477  0.150204  0.113323   
none        0.122469  0.126829  0.139818  0.129824  0.151054  0.113040   
tick        0.123876  0.125990  0.139215  0.130623  0.150268  0.113584   

                                
pred_label      none      tick  
label                           
ant         0.096374  0.120082  
bedbug      0.096038  0.120469  
bee         0.096499  0.120508  
horsefly    0.096002  0.120154  
mite        0.095808  0.120584  
mosquito    0.096373  0.120294  
none        0.096936  0.120029  
tick        0.096159  0.120286

In [14]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).sort_values(["img_path", "pred_prob"], ascending=False).groupby(["img_path", "label"]).first()

,,pred_label,pred_prob
img_path,label,,
../data/cleaned/ant/0161d75756b770ac6fd28e6b5a3156a201ae179a.jpg,ant,mite,0.151958
../data/cleaned/ant/06443ff82c28fd0d52f62868d8995e8e70107d7a.jpg,ant,mite,0.151467
../data/cleaned/ant/1018442462dbb9419f327b652ff28a0c55a2710d.jpg,ant,mite,0.154454
../data/cleaned/ant/11ad398be6c87a2a6fca4de0b332022f12521cb8.jpg,ant,mite,0.148712
../data/cleaned/ant/477e86816e544bcf371c470b65c887567f8fd186.jpg,ant,mite,0.150962
...,...,...,...
../data/cleaned/tick/b4f637d0ccc21bb6631b77c6bfe1f8fff2e8cf06.jpg,tick,mite,0.148771
../data/cleaned/tick/c6d142ee77529c16153491fecb202e2edd8cefec.jpg,tick,mite,0.148703
../data/cleaned/tick/d3eb4c5c6aa879b6a460910d461b9a22690105af.jpg,tick,mite,0.149769
